In [206]:
import mmap
import numpy as np
import binascii
import hashlib
import base58
import struct
import datetime
import hashlib

In [209]:
def publicKeyDecode(pub):
    if pub.lower().startswith('76a914'):
        pub = pub[6:-4]
        result = (b'\x00') + binascii.unhexlify(pub)
        h5 = hashlib.sha256(result)
        h6 = hashlib.sha256(h5.digest())
        result += h6.digest()[:4]
        return base58.b58encode(result)
    elif pub.lower().startswith('a9'):
        return ""
    elif startsWithOpNCode(pub):
        pub = pub[2:-2]
        h3 = hashlib.sha256(binascii.unhexlify(pub))
        h4 = hashlib.new('ripemd160', h3.digest())
        result = (b'\x00') + h4.digest()
        h5 = hashlib.sha256(result)
        h6 = hashlib.sha256(h5.digest())
        result += h6.digest()[:4]
        return base58.b58encode(result)
    return ""

In [193]:
def reverse(h):
    byte_array = bytearray(h)
    h_new = bytearray(b'0')
    for i in range(0,len(byte_array),2):
        h_new[i:i+1] = byte_array[len(byte_array)-1-i-1], byte_array[len(byte_array)-1-i]
        
    return bytes(h_new)

In [194]:
def hexToInt(value):
    return int(binascii.hexlify(value), 16)

def hexToStr(value):
    return binascii.hexlify(value)

def readVarInt(blockFile):
    varInt = ord(blockFile.read(1))
    returnInt = 0
    if varInt < 0xfd:
        return varInt
    if varInt == 0xfd:
          returnInt = readShortLittleEndian(blockFile)
    if varInt == 0xfe:
        returnInt = readIntLittleEndian(blockFile)
    if varInt == 0xff:
        returnInt = readLongLittleEndian(blockFile)
    return int(binascii.hexlify(returnInt), 16)

In [195]:
def stringLittleEndianToBigEndian(string):
    string = binascii.hexlify(string)
    n = len(string) / 2
    fmt = '%dh' % n
    return struct.pack(fmt, *reversed(struct.unpack(fmt, string)))

def readIntLittleEndian(blockFile):
    return struct.pack(">I", struct.unpack("<I", blockFile.read(4))[0])

def readShortLittleEndian(blockFile):
    return struct.pack(">H", struct.unpack("<H", blockFile.read(2))[0])

def readLongLittleEndian(blockFile):
    return struct.pack(">Q", struct.unpack("<Q", blockFile.read(8))[0])


In [196]:
def readInput(blockFile):
    previousHash = binascii.hexlify(blockFile.read(32)[::-1])
    outId = binascii.hexlify(readIntLittleEndian(blockFile))
    scriptLength = readVarInt(blockFile)
    scriptSignatureRaw = hexToStr(blockFile.read(scriptLength))
    scriptSignature = scriptSignatureRaw
    seqNo = binascii.hexlify(readIntLittleEndian(blockFile))

    print("\n" + "Input")
    print("-" * 20)
    print("> Previous Hash: ", previousHash)
    print("> Out ID: ", outId)
    print("> Script length: " + str(scriptLength))
    print("> Script Signature (PubKey) Raw: ", scriptSignatureRaw)
    print("> Script Signature (PubKey): ", scriptSignature)
    print("> Seq No: ", seqNo)

In [204]:
def readOutput(blockFile):
    value = hexToInt(readLongLittleEndian(blockFile)) / 100000000.0
    scriptLength = readVarInt(blockFile)
    scriptSignatureRaw = hexToStr(blockFile.read(scriptLength))
    scriptSignature = scriptSignatureRaw
    address = ''
    try:
        address = publicKeyDecode(scriptSignature)
    except:
        address = ''
  
    print("\n" + "Output")
    print("-" * 20)
    print("> Value: " + str(value))
    print("> Script length: " + str(scriptLength))
    print("> Script Signature (PubKey) Raw: ", scriptSignatureRaw)
    print("> Script Signature (PubKey): ", scriptSignature)
    print("> Address: " + address)

In [201]:
def readTransaction(blockFile):
    extendedFormat = False
    beginByte = blockFile.tell()
    inputIds = []
    outputIds = []
    version = hexToInt(readIntLittleEndian(blockFile)) 
    cutStart1 = blockFile.tell()
    cutEnd1 = 0
    inputCount = readVarInt(blockFile)
    
    print("\n\n" + "Transaction")
    print("-" * 100)
    print("Version: " + str(version))
    
    print("\nInput Count: " + str(inputCount))
    for inputIndex in range(0, inputCount):
        inputIds.append(readInput(blockFile))
    outputCount = readVarInt(blockFile)
    print("\nOutput Count: " + str(outputCount))
    for outputIndex in range(0, outputCount):
        outputIds.append(readOutput(blockFile))
        
    lockTime = hexToInt(readIntLittleEndian(blockFile))
    if lockTime < 500000000:
        print("\nLock Time is Block Height: " + str(lockTime))
    else:
        print("\nLock Time is Timestamp: " + datetime.datetime.fromtimestamp(lockTime).strftime('%d.%m.%Y %H:%M'))


In [202]:
def process_block(blockFile):
    magicNumber = binascii.hexlify(blockFile.read(4))
    blockSize = hexToInt(readIntLittleEndian(blockFile))
    version = hexToInt(readIntLittleEndian(blockFile))
    previousHash = reverse(binascii.hexlify(blockFile.read(32)))
    merkleHash = reverse(binascii.hexlify(blockFile.read(32)))
    creationTimeTimestamp = hexToInt(readIntLittleEndian(blockFile))
    creationTime = datetime.datetime.fromtimestamp(creationTimeTimestamp).strftime('%d.%m.%Y %H:%M')
    bits = hexToInt(readIntLittleEndian(blockFile))
    nonce = hexToInt(readIntLittleEndian(blockFile))
    countOfTransactions = readVarInt(blockFile)
    
    print("Magic Number: ", magicNumber)
    print("Blocksize: " + str(blockSize))
    print("Version: " + str(version))
    print("Previous Hash: ", previousHash)
    print("Merkle Hash: ", merkleHash)
    print("Time: " , creationTime)
    print("Bits: " + str(bits))
    print("Nonce: " + str(nonce))
    print("Count of Transactions: " + str(countOfTransactions))
    
    for transactionIndex in range(0, countOfTransactions):
        readTransaction(blockFile)

In [210]:
with open('blk00000.dat', 'rb') as file:
    file.read(293)
    process_block(file)

Magic Number:  b'f9beb4d9'
Blocksize: 215
Version: 1
Previous Hash:  b'000000000019d6689c085ae165831e934ff763ae46a2a6c172b3f1b60a8ce26f'
Merkle Hash:  b'0e3e2357e806b6cdb1f70b54c3a3a17b6714ee1f0e68bebb44a74b1efd512098'
Time:  09.01.2009 03:54
Bits: 486604799
Nonce: 2573394689
Count of Transactions: 1


Transaction
----------------------------------------------------------------------------------------------------
Version: 1

Input Count: 1

Input
--------------------
> Previous Hash:  b'0000000000000000000000000000000000000000000000000000000000000000'
> Out ID:  b'ffffffff'
> Script length: 7
> Script Signature (PubKey) Raw:  b'04ffff001d0104'
> Script Signature (PubKey):  b'04ffff001d0104'
> Seq No:  b'ffffffff'

Output Count: 1

Output
--------------------
> Value: 50.0
> Script length: 67
> Script Signature (PubKey) Raw:  b'410496b538e853519c726a2c91e61ec11600ae1390813a627c66fb8be7947be63c52da7589379515d4e0a604f8141781e62294721166bf621e73a82cbf2342c858eeac'
> Script Signature (PubKe

In [191]:
with open('blk00000.dat', 'rb') as file:
    file.read(293)
    print(file.tell())

293


In [ ]:
|